<a href="https://colab.research.google.com/github/GisstHubana/FHIR_Reporting_Tool/blob/GisstHubana-patch-html/Code/fhir_reporting_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vorbereitung
## CleanUp - Eventuell gespeicherte Variablen löschen vor Ausführung des Colab books für eine bessere Reproduzierbarkeit

In [ ]:
%reset -f

## Installieren notwendiger Pakete

In [ ]:
!pip install pyngrok
!pip install flask-ngrok
!pip install pandas

## gnrok mit Authorizierungstoken verknüpfen

In [ ]:
!ngrok authtoken 2Wlt2vGzytTpJ6zABKGKKwfiTof_7y4sewzGkNGy7zW6GB5oz #notwendig für späteren Aufruf der Webseite

Importieren der notwendigen Libraries

In [ ]:
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok
import secrets
import sqlite3
import json
import pandas as pd
import glob
import requests

## Abfrage des Blaze-Servers

In [ ]:
url = "https://mii-agiop-3p.life.uni-leipzig.de/blaze"
# Verbindung mit url und Datenabfrage
r = requests.get(url)
data = r.json()

In [ ]:
# Anzeigen der Keys
data.keys()

In [ ]:
len(data["entry"])

In [ ]:
# Zugriff auf den Wert des Schlüssels 'type'
type_value = data['type']

# Ausgabe der Variablen
print(type_value)

In [ ]:
# Zugriff auf den Wert des Schlüssels 'type'
entry_value = data['entry']

# Ausgabe der Variablen
print(entry_value)

In [ ]:
test = "https://mii-agiop-3p.life.uni-leipzig.de/blaze/Patient?_format=xml"

print(test)

Erstellung einer virtuellen Kopie des GitHub Repositorium FHIR_Reporting_Tool in Colab

In [ ]:
%cd /content # Wechsel in den Ordner /content - für den Fall, dass in einem Unterordner gearbeitet wurde

!git clone https://github.com/GisstHubana/FHIR_Reporting_Tool.git #virtuelle Kopie des Repositoriums - Bei Neustart zuvor Runtime/disconnect and delete runtime ausführen
!ls # Auflistung aller Ordnerinhalte

In [ ]:
!git clone https://github.com/medizininformatik-initiative/kerndatensatz-testdaten.git #virtuelle Kopie des Medizininformatik-Initiative-Repositoriums - Bei Neustart zuvor Runtime/disconnect and delete runtime ausführen

### Database in Flask

In [ ]:
db_folder = "/content/FHIR_Reporting_Tool/" # Benennung des Datenbank-Ordnerts
conn = sqlite3.connect(db_folder+"mydatabase.db") # falls Datenbank noch nicht existiert wird sie mit diesem Aufruf virtuell erzeugt
cursor = conn.cursor() # Cursor erstellen

## Einlesen als ndjson-Datei

In [ ]:
# Auslesen des Inhalts einer ndjason-Datei

# Pfadangabe der NDJSON-Datei
ndjson_file = "/content/kerndatensatz-testdaten/Test_Data/Vorhofflimmern/VHF-Testdaten_01-VHF00001-VHF01000.ndjson"

In [ ]:
# Ein leeres Set zum Sammeln eindeutiger Schlüssel aus allen JSON-Objekten
alle_schluessel = set()

# Öffnen der NDJSON-Datei im Lesemodus
with open(ndjson_file, "r") as file:
    # Durchlaufen Sie die Zeilen der Datei
    for line in file:
        # Jede Zeile enthält ein JSON-Objekt, deshalb laden wir es
        data = json.loads(line)

        # Sammeln aller Schlüssel (Variablen) aus dem aktuellen JSON-Objekt
        aktuelle_schluessel = set(data.keys())

        # Einfügen gefundener Schlüssel Set alle_schluessel
        alle_schluessel.update(aktuelle_schluessel)

# Ausgabe aller eindeutiger Schlüssel aus allen JSON-Objekten der NDJSON-Datei
print("Alle gefundenen Schlüssel:")
for schluessel in alle_schluessel:
    print(schluessel)

In [ ]:
# Dateiname der NDJSON-Datei
ndjson_file = "/content/kerndatensatz-testdaten/Test_Data/Vorhofflimmern/VHF-Testdaten_01-VHF00001-VHF01000.ndjson"

# Anzahl der Zeilen, die Sie ausgeben möchten
anzahl_zeilen = 5  # Ändern Sie dies auf die gewünschte Anzahl

# Öffnen der NDJSON-Datei im Lesemodus
with open(ndjson_file, "r") as file:
    # Durchlaufen Sie die ersten N Zeilen der Datei
    for i, line in enumerate(file):
        if i >= anzahl_zeilen:
            break


        # Jede Zeile enthält ein JSON-Objekt, deshalb laden wir es
        data = json.loads(line)

        # Hier können Sie auf die Daten im JSON-Objekt zugreifen
        # Zum Beispiel, wenn jedes JSON-Objekt eine "name" Eigenschaft hat
        entry = data.get("entry")

        # Verarbeiten Sie die Daten nach Bedarf
        print("entry:", entry)

In [ ]:
# ndjson-Datei als dataframe einlesen:
data_list_ndjson = [] # Liste, um die JSON-Objekte aus der NDJSON-Datei zu speichern

# Die NDJSON-Datei Zeile für Zeile lesen und JSON-Objekte parsen
with open(ndjson_file, 'r') as file:
    for line in file:
        data = json.loads(line)
        data_list_ndjson.append(data)

ndjson_df = pd.DataFrame(data_list_ndjson)

# print(ndjson_df.head(ndjson_df))
ndjson_df.iloc[0:5]

## Einlesen als Excel-Datei

In [ ]:
# Pfade zur Excel-Datei und zum Tabellenblatt
excel_datei_pfad = '/content/kerndatensatz-testdaten/Test_Data/Vorhofflimmern/VHF-Testdaten_01.xlsx'  # Passe den Pfad zur Excel-Datei an

# Excel-Datei einlesen
vhf_test_df = pd.read_excel(excel_datei_pfad) # Passe den Namen des Tabellenblatts an

# Header ausgeben
header = vhf_test_df.columns.tolist()
print("Header der Excel-Datei:")
print(header)
print(vhf_test_df.head(5))

In [ ]:
#query
create_table_query = """
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  username TEXT NOT NULL,
  password TEXT NOT NULL

)

"""
cursor.execute(create_table_query)

insert_query = """
INSERT INTO users(username, password)
SELECT ?, ?
WHERE NOT EXISTS (
  SELECT 1 FROM users WHERE username = ?

)
"""
user_data = ("USER3", "1234", "USER3")
user_data = ("Tatjana", "1234", "Tatjana")
cursor.execute(insert_query, user_data)
conn.commit()

select_query = "SELECT * FROM users"
cursor.execute(select_query)
rows = cursor.fetchall()
for row in rows:
  print(row)

### Abfragebeispiele auf die Dataframes ndjson_df und vhf_test_df

In [ ]:
ergebnis_df = pd.DataFrame(ndjson_df["type"].tolist())
print(ergebnis_df.head(5))

In [ ]:
ergebnis_df = pd.DataFrame(ndjson_df["entry"].tolist())
print(ergebnis_df.head(5))

In [ ]:
ergebnis_df = pd.DataFrame(vhf_test_df["Patient-ID"].tolist())
print(ergebnis_df.head(5))

In [ ]:
ergebnis_df = pd.DataFrame(vhf_test_df["Geburtsdatum"].tolist())
print(ergebnis_df.head(5))

In [ ]:
# Die Spaltenüberschriften (Keys) des Dataframes vhf_test_df ausgeben
column_names_vhf = vhf_test_df.columns
print(column_names_vhf)

In [ ]:
# print((vhf_test_df.to_html()).head(5)) # String Objekt daher print head nicht möglich und sonst zu groß
print(type(vhf_test_df.to_html()))

In [ ]:
print(vhf_test_df.head(5))

In [ ]:
# template_folder = "/content/FHIR_Reporting_Tool/templates"
# static_folder = "/content/FHIR_Reporting_Tool/static"
# db_folder = "/content/FHIR_Reporting_Tool/"
# app = Flask(__name__, template_folder=template_folder, static_folder=static_folder) # Flask-App, Benennung template-Ordner
# app.secret_key = secrets.token_hex(32)
# run_with_ngrok(app) # run_with_ngrok Funktion hinzufügen, um App mit ngrok auszuführen

# # conn = sqlite3.connect(db_folder+"mydatabase.db") # falls Datenbank noch nicht existiert wird sie mit diesem Aufruf erzeugt
# # cursor = conn.cursor()

# @app.route('/') # Erstellung einer Routendefinition
# def index():
#     return render_template("fhir_abfrage.html")

# @app.route('/table')
# def show_table():
#     # Konvertieren des DataFrames in HTML
#     table = vhf_test_df.to_html()

#     # Übergab des HTMLs an das Render-Template
#     return render_template('result_fhir.html', table=table_html)

# @app.route('/abfrage', methods=['POST']) # Erstellung einer Routendefinition Abfrage
# def abfrage():
#     abfrage = request.form['abfrage']

#     # Führen Sie die Abfrage auf dem DataFrame durch, z.B. Filtern oder Aggregieren der Daten.
#     # ergebnis_df = pd.DataFrame(vhf_test_df[abfrage].tolist()) # Abfrage Excel_VHF_Test_Dataframe
#     ergebnis_df = pd.DataFrame(vhf_test_df[abfrage].tolist()) # Abfrage ndjson_VHF_Test_Dataframe

#     # Konvertieren Sie das Ergebnis-DataFrame in HTML.
#     # ergebnis_html = ergebnis_df.to_html(classes='table table-bordered table-hover')
#     ergebnis_html = ergebnis_df.to_html()
#     return render_template('fhir_ergebnis.html', ergebnis_html=ergebnis_html)

# # @app.route('/ergebnis', methods=['POST']) # Erstellung einer Routendefinition Ergebnis
# # def verarbeitung():
# #     eingabe = request.form['eingabe']
# #     umgekehrt = eingabe[::-1]
# #     return render_template('fhir_ergebnis.html', eingabe=eingabe, umgekehrt=umgekehrt)

# if __name__ == '__main__': # Starten der Flask-Anwendung
#   app.run()

In [ ]:
template_folder = "/content/FHIR_Reporting_Tool/templates"
static_folder = "/content/FHIR_Reporting_Tool/static"
db_folder = "/content/FHIR_Reporting_Tool/"
app = Flask(__name__, template_folder=template_folder, static_folder=static_folder) # Flask-App, Benennung template-Ordner
# app.debug = True # Debug-Modus aktivieren - webseite lädt nicht wenn eingeschaltet
app.secret_key = secrets.token_hex(32)
run_with_ngrok(app) # run_with_ngrok Funktion hinzufügen, um App mit ngrok auszuführen

@app.route('/', methods=["GET", "POST"])
def login():

  if request.method == "POST":
    username = request.form.get("username")
    password = request.form.get("password")

    conn = sqlite3.connect(db_folder+"mydatabase.db")
    cursor = conn.cursor()
    # query
    query = """
    SELECT * FROM users WHERE username = ? AND password = ?
    """
    cursor.execute(query, (username, password))
    result = cursor.fetchone()

    if result:
     return render_template("eingeloggt.html", name=username)
    else:
      message = "Wrong username or password. Please try again!"
      return render_template("login.html", message=message)

  else:
    return render_template("login.html")

@app.route('/home') # Erstellung einer Routendefinition home
def index():
    return render_template("index.html")

@app.route('/eingeloggt')
def eingeloggt():

    return render_template("eingeloggt.html", name=username)

@app.route('/xls_abfrage') # Erstellung einer Routendefinition
def abfrage():
    return render_template("fhir_abfrage.html")

@app.route('/xls_abfrage', methods=['POST']) # Erstellung einer Routendefinition Abfrage
def xls_abfrage():
    abfrage = request.form['xls_abfrage']

    # Abfrage auf dem aus xls genererierten DataFrame
    ergebnis_df = pd.DataFrame(vhf_test_df[abfrage].tolist()) # Abfrage Excel_VHF_Test_Dataframe

    # Konvertieren des Ergebnis-DataFrames in HTML
    ergebnis_html = ergebnis_df.to_html()
    return render_template('fhir_ergebnis.html', ergebnis_html=ergebnis_html)

@app.route('/blaze_abfrage') # Erstellung einer Routendefinition
def blaze_abfrage():
    return render_template("fhir_abfrage.html")

@app.route('/json_abfrage') # Erstellung einer Routendefinition
def json_abfrage():
    return render_template("fhir_abfrage.html")

@app.route('/table')
def show_table():
    # HTML-String definieren, muss nicht umgewandelt werden, das es schon als html-String vorliegt
    table_html = vhf_test_df

    # # Konvertieren des DataFrames in HTML hinfällig
    # table_html = df.to_html()

    # Übergeben Sie das HTML an das Render-Template
    return render_template('result_fhir.html', table=table_html)

# @app.route('/page3')
# def page3():
#     return render_template("page3.html")

app.run()

In [ ]:
# Text in Zeilenumbrüche teilen
zeilen = (vhf_test_df.to_html()).split('\n')
# Ausgabe der ersten 120 Zeilen
for zeile in zeilen[:120]:
    print(zeile)

## Abschluss - Runtime löschen und abschließend clear all outputs

In [ ]:
# # Bei Neustart zuvor Runtime/disconnect and delete runtime ausführen
# from google.colab import runtime
# runtime.unassign() # delete runtime